In [1]:
import numpy as np
import pylab as pl
import scipy.fftpack as ff
import h5py

In [2]:
pl.rcParams['figure.figsize']  = 12, 7.5
pl.rcParams['lines.linewidth'] = 1.5
pl.rcParams['font.family']     = 'serif'
pl.rcParams['font.weight']     = 'bold'
pl.rcParams['font.size']       = 20
pl.rcParams['font.sans-serif'] = 'serif'
pl.rcParams['text.usetex']     = True
pl.rcParams['axes.linewidth']  = 1.5
pl.rcParams['axes.titlesize']  = 'medium'
pl.rcParams['axes.labelsize']  = 'medium'

pl.rcParams['xtick.major.size'] = 8
pl.rcParams['xtick.minor.size'] = 4
pl.rcParams['xtick.major.pad']  = 8
pl.rcParams['xtick.minor.pad']  = 8
pl.rcParams['xtick.color']      = 'k'
pl.rcParams['xtick.labelsize']  = 'medium'
pl.rcParams['xtick.direction']  = 'in'

pl.rcParams['ytick.major.size'] = 8
pl.rcParams['ytick.minor.size'] = 4
pl.rcParams['ytick.major.pad']  = 8
pl.rcParams['ytick.minor.pad']  = 8
pl.rcParams['ytick.color']      = 'k'
pl.rcParams['ytick.labelsize']  = 'medium'
pl.rcParams['ytick.direction']  = 'in'

### For reference
\begin{align}
\hat{V}(k) &= \int_{0}^{1} V(x)e^{-2\pi\;i\;k\;x}dx \\ \\
V(x) &= \frac{1}{Npoints}\int_{0}^{1} \hat{V}(k)e^{+2\pi\;i\;k\;x}dk \\ \\
\hat{V}(k) &= \frac{1}{4\pi^{2}\;k^2}\hat{\rho(k)} \\ \\
\hat{E}(k) &= -i(2\pi\;k)\hat{V}(k)
\end{align}

In [3]:
# FFT solver :
def fft_poisson(rho,dx):

    kspace = ff.fftfreq(len(rho), d = dx)
    kspace[0] = 0.0001
    rho_kspace = ff.fft(rho)

    V_kspace = np.zeros(len(rho))
    
    V_kspace[1:] =  (1/(4 * np.pi**2 * kspace[1:]**2)) * rho_kspace[1:]
    V_kspace[0]  =  (1/(4 * np.pi**2)) * np.sum(rho)/(len(rho)) 
    
    E_kspace =  -1j * 2 * np. pi * kspace * V_kspace
    
    V = ff.ifft(V_kspace)

    V = V.astype(np.double)
    
    E = ff.ifft(E_kspace)
    
    E = E.astype(np.double)
    
    return V, E

In [4]:
# b1 charge depositor
def cloud_charge_deposition(charge, zone_x, frac_x, x_grid):

    left_corner_charge = (1 - frac_x) * charge
    right_corner_charge = (frac_x) * charge

    left_corners  = zone_x.copy()
    right_corners = left_corners + 1

    corners = np.concatenate([left_corners, right_corners], axis=0)
    charges = np.concatenate([left_corner_charge, right_corner_charge], axis=0)

    rho, temp = np.histogram(corners, bins=len(x_grid), range=(0, len(x_grid)), weights=charges)

    return rho

In [5]:
k_boltzmann     = 1
mass_electron   = 1
tempertature    = 1
charge_electron = -1
charge_ion      = +1

In [6]:
length_domain_x = 1

In [7]:
number_of_electrons = 3

positions_x = length_domain_x * np.random.rand(number_of_electrons)

mu, sigma = 0, (k_boltzmann * tempertature / mass_electron)

velocity_x = np.random.normal(mu, sigma, number_of_electrons)

In [8]:
divisions_domain_x = 100

x_grid = np.linspace(0, length_domain_x, divisions_domain_x + 1, endpoint=True)

dx = x_grid[1] - x_grid[0]

In [9]:
start_time = 0

end_time   = 3

dt  = 0.001

time = np.arange(start_time, end_time + dt, dt)

In [10]:
rho_ions = (charge_ion * number_of_electrons) / (length_domain_x)

In [11]:
for time_index in range(len(time)):
    print('Computing for time_index = ', time_index)
    # Updating the positions
    print('positions_x are ',positions_x)
    positions_x += velocity_x * dt

    # Boundary conditions
    outside_domain = np.where([positions_x < 0])[0]

    positions_x[outside_domain] = positions_x[outside_domain] + length_domain_x

    outside_domain = np.where([positions_x > length_domain_x])[0]
    positions_x[outside_domain] -= length_domain_x

    # Finding interpolant fractions for the positions

    zone_x = np.floor(((positions_x - x_grid[0]) / dx))
    zone_x = zone_x.astype(np.int)
    frac_x = (positions_x - x_grid[zone_x]) / (dx)

    # Charge deposition using linear weighting scheme

    rho = cloud_charge_deposition(charge_electron, zone_x, frac_x, x_grid)
    rho+= rho_ions

    # Calculating the potential/Electric field from the charge deposition.

    V, Ex = fft_poisson(rho,dx)
    

    # Interpolating the fields at each particle
    print('zone_x is ', zone_x)
    print('zone_x + 1 is ', zone_x + 1)
    Ex_particle = Ex[zone_x] + frac_x * Ex[zone_x + 1]
#     Ex_particle = np.reshape(Ex_particle, (len(Ex_particle)))

    # Updating the particles using the interpolated field values.

    velocity_x += (Ex_particle * charge_electron / mass_electron ) * dt

#     input('check')

    h5f = h5py.File('data/timestepped_data/solution_'+str(time_index)+'.h5', 'w')
    h5f.create_dataset('positions_x',   data = positions_x)
    h5f.create_dataset('velocity_x',   data = velocity_x)
    h5f.create_dataset('Ex',   data = (Ex))
    h5f.close()

Computing for time_index =  0
positions_x are  [ 0.46194579  0.14784864  0.20006871]
zone_x is  [46 14 20]
zone_x + 1 is  [47 15 21]
Computing for time_index =  1
positions_x are  [ 0.46127603  0.14707293  0.20084688]
zone_x is  [46 14 20]
zone_x + 1 is  [47 15 21]
Computing for time_index =  2
positions_x are  [ 0.46060626  0.14629721  0.20162505]
zone_x is  [45 14 20]
zone_x + 1 is  [46 15 21]
Computing for time_index =  3
positions_x are  [ 0.4599365   0.1455215   0.20240322]
zone_x is  [45 14 20]
zone_x + 1 is  [46 15 21]
Computing for time_index =  4
positions_x are  [ 0.45926673  0.14474577  0.2031814 ]
zone_x is  [45 14 20]
zone_x + 1 is  [46 15 21]
Computing for time_index =  5
positions_x are  [ 0.45859696  0.14397005  0.20395957]
zone_x is  [45 14 20]
zone_x + 1 is  [46 15 21]
Computing for time_index =  6
positions_x are  [ 0.45792718  0.14319432  0.20473775]
zone_x is  [45 14 20]
zone_x + 1 is  [46 15 21]
Computing for time_index =  7
positions_x are  [ 0.4572574   0.142418

/home/tejas/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
/home/tejas/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:17: ComplexWarning: Casting complex values to real discards the imaginary part
/home/tejas/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:21: ComplexWarning: Casting complex values to real discards the imaginary part



zone_x is  [42 10 24]
zone_x + 1 is  [43 11 25]
Computing for time_index =  61
positions_x are  [ 0.42108623  0.10052503  0.2475384 ]
zone_x is  [42  9 24]
zone_x + 1 is  [43 10 25]
Computing for time_index =  62
positions_x are  [ 0.42041636  0.09974916  0.24831659]
zone_x is  [41  9 24]
zone_x + 1 is  [42 10 25]
Computing for time_index =  63
positions_x are  [ 0.41974648  0.09897329  0.24909479]
zone_x is  [41  9 24]
zone_x + 1 is  [42 10 25]
Computing for time_index =  64
positions_x are  [ 0.41907661  0.09819742  0.24987297]
zone_x is  [41  9 25]
zone_x + 1 is  [42 10 26]
Computing for time_index =  65
positions_x are  [ 0.41840673  0.09742154  0.25065116]
zone_x is  [41  9 25]
zone_x + 1 is  [42 10 26]
Computing for time_index =  66
positions_x are  [ 0.41773685  0.09664567  0.25142935]
zone_x is  [41  9 25]
zone_x + 1 is  [42 10 26]
Computing for time_index =  67
positions_x are  [ 0.41706696  0.09586979  0.25220753]
zone_x is  [41  9 25]
zone_x + 1 is  [42 10 26]
Computing for

IndexError: index 101 is out of bounds for axis 1 with size 101

In [ ]:
# # cloud_charge_deposition test
# charge = 1
# x_grid = np.array([0, 0.2, 0.4, 0.6, 0.8, 1.0])
# x = np.array([0.9])
# zone_x = np.array([4])
# frac_x = np.array([0.5])
# print(cloud_charge_deposition(charge, zone_x, frac_x, x_grid))

In [ ]:
# # # FFT test

# rho_size = 200
# x = np.linspace(0, 1, rho_size )

# A = 0.1
# rho_0 = 0
# rho =  rho_0 + A * np.cos(2 * np.pi * x)
# dx = x[1] - x[0]

# V, E = fft_poisson(rho, dx)
# print('max(V)', max(V))
# # pl.plot(x, V,'--',lw = 3,label = '$V$')
# # pl.plot(x, ff.ifft(ff.fft(rho)), label = 'ifft(fft(rho))')
# # pl.plot(x, rho, label = r'$\rho$')
# # pl.plot(x, np.cos(2*np.pi*x)/(4 * np.pi**2), label= '$\mathrm{Analytical\;V}$')
# pl.plot(x, E, label = '$E_{x}$')
# pl.plot(x, A * np.sin(2 * np.pi * x)/(2 * np.pi), label = '$E_{x}\;analytical$')
# # pl.ylim(0,1)
# pl.legend()
# pl.show()
# pl.clf()